In [1]:
using Plots, TaylorSeries
pyplot(legend = :false)

Plots.PyPlotBackend()

In [2]:
const λ = 1.0
const Z = 2.0
const _ordenTaylor = 20
const _epsAbs = 1.0e-20

1.0e-20

In [3]:
# Returns stepsize of the integration and a vector with the updated values of the dependent
# variables
function taylorStepper{T<:Real}( jetEqs::Function, vec0::Array{T,1} )
    
    n = length( vec0 )

    vec0T = Array(Taylor1{T},n)
    @simd for i in eachindex(vec0)
        @inbounds vec0T[i] = Taylor1([vec0[i]], _ordenTaylor)
    end

    # Jets
    vec1T = jetEqs( vec0 )
    
    # Step-size
    hh = Inf
    for i in eachindex(vec1T)
        @inbounds h1 = stepsize( vec1T[i], _epsAbs )
        hh = min( hh, h1 )
    end
    
    # Values at t0+h
    @simd for i in eachindex(vec0)
        @inbounds vec0[i] = evaluate( vec1T[i], hh )
    end
    
    return hh, vec0
end

# Returns the maximum step size from epsilon and the last two coefficients of the x-Taylor series 
function stepsize{T<:Real}(x::Taylor1{T}, epsilon::Float64)
    ord = x.order
    h = Inf
    for k in [ord-1, ord]
        kinv = 1.0/k
        aux = abs( x.coeffs[k+1] )
        h = min(h, (epsilon/aux)^kinv)
    end
    return h
end

stepsize (generic function with 1 method)

Aquí implementaremos el hamiltoniano regularizado, con $E = -1$, debe cumplirse que $H = 0 \ \forall t$, primero, tomaremos una función auxiliar:

In [4]:
ks(Q) = Q^2

ks (generic function with 1 method)

In [5]:
function regHam(Q₁, Q₂, P₁, P₂)
    #Cantidades auxiliares
    P₁² = P₁^2
    P₂² = P₂^2
    Q₁² = Q₁^2
    Q₂² = Q₂^2
    nf = abs(ks(Q₁) - ks(Q₂))
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end


regHam (generic function with 2 methods)

Implementaremos también la función para regresar las velocidades al espacio fase de la configuración original... 

In [6]:
function ksv(Q, P)
    v₁ = (Q*P)/(2*Q^2)
    return v₁
end

ksv (generic function with 1 method)

In [7]:
function jetundriven{T<:Real}( vec::Array{T,1} )

    tT = Taylor1(vec[1], _ordenTaylor)
    Q1T = Taylor1(vec[2], _ordenTaylor)
    P1T = Taylor1(vec[3], _ordenTaylor)
    Q2T = Taylor1(vec[4], _ordenTaylor)
    P2T = Taylor1(vec[5], _ordenTaylor)
    
    # Auxiliary quantities
    sQ1 = zeros( T, _ordenTaylor+1 )
    sP1 = zeros( T, _ordenTaylor+1 )
    sQ2 = zeros( T, _ordenTaylor+1 )
    sP2 = zeros( T, _ordenTaylor+1 )
    f1 = zeros( T, _ordenTaylor+1 )
    f12 = zeros( T, _ordenTaylor+1 )
    nfQ = zeros( T, _ordenTaylor+1 )
    rnfQ = zeros( T, _ordenTaylor+1 )
    n3fQ = zeros( T, _ordenTaylor+1 )

    dotQ1 = zeros( T, _ordenTaylor+1 )
    dotP1 = zeros( T, _ordenTaylor+1 )
    dotQ2 = zeros( T, _ordenTaylor+1 )
    dotP2 = zeros( T, _ordenTaylor+1 )
        
    s11P1 = zeros( T, _ordenTaylor+1 )
    s1P1 = zeros( T, _ordenTaylor+1 )
    P1d1 = zeros( T, _ordenTaylor+1 )
    s12P1 = zeros( T, _ordenTaylor+1 )
    s2m2P1 = zeros( T, _ordenTaylor+1 )
    P1d2 = zeros( T, _ordenTaylor+1 )
    s2P1 = zeros( T, _ordenTaylor+1 )
    
    
    s11P2 = zeros( T, _ordenTaylor+1 )
    P2d1 = zeros( T, _ordenTaylor+1 )
    s1P2 = zeros( T, _ordenTaylor+1 )
    s12P2 = zeros( T, _ordenTaylor+1 )
    s2m2P2 = zeros( T, _ordenTaylor+1 )
    P2d2 = zeros( T, _ordenTaylor+1 )
    s2P2 = zeros( T, _ordenTaylor+1 )
    
    # Now the implementation
    for k = 0:_ordenTaylor-1
        knext = k+1
        # The right-hand size of the eqs of motion
        # This is more adpated for this problem, and avoids many superflous operations
        #@show "That's ok"
        sQ1[knext] = TaylorSeries.squareHomogCoef(k, Q1T.coeffs)
        sP1[knext] = TaylorSeries.squareHomogCoef(k, P1T.coeffs)
        sQ2[knext] = TaylorSeries.squareHomogCoef(k, Q2T.coeffs)
        sP2[knext] = TaylorSeries.squareHomogCoef(k, P2T.coeffs)
        
        f1[knext] = sQ1[knext] - sQ2[knext]  
        f12[knext] = TaylorSeries.squareHomogCoef(k, f1)
        nfQ[knext] = TaylorSeries.powHomogCoef(k, f12, 0.5, nfQ, 0)
        n3fQ[knext] = TaylorSeries.powHomogCoef(k, f12, 1.5, n3fQ, 0)
        n2Qs[knext] = TaylorSeries.mulHomogCoef(k, sQ1, sQ2)
        
        P1d1[knext] = TaylorSeries.divHomogCoef(k, sQ2, nfQ, P1d1, 0)
        s11P1[knext] = -0.125*sP2[knext] + Z - P1d1[knext] - sQ2[knext]
        s1P1[knext] = 2.0*TaylorSeries.mulHomogCoef(k, Q1T.coeffs, s11P1)
        P1d2[knext] = TaylorSeries.divHomogCoef(k, f1, n3fQ, P1d2, 0)
        s2m2P1[knext] = 2*TaylorSeries.mulHomogCoef(k, P1d2, Q1T.coeffs)
        s2P1[knext] = TaylorSeries.mulHomogCoef(k, n2Qs, s2m2P1)
        
        
        
        P2d1[knext] = TaylorSeries.divHomogCoef(k, sQ1, nfQ, P2d1, 0)
        s11P2[knext] = -0.125*sP1[knext] + Z - P2d1[knext] - sQ1[knext]
        s1P2[knext] = 2.0*TaylorSeries.mulHomogCoef(k, Q2T.coeffs, s11P2)
        s2m2P2[knext] = -2*TaylorSeries.mulHomogCoef(k, P1d2, Q21T.coeffs)
        s2P2[knext] = TaylorSeries.mulHomogCoef(k, n2Qs, s2m2P2)
        
    
        dotQ1[knext] = 0.25*TaylorSeries.mulHomogCoef(k,sQ2, P1T.coeffs)
        dotP1[knext] = s1P1[knext] + s2P1[knext]
        dotQ2[knext] = 0.25*TaylorSeries.mulHomogCoef(k,n2Q1, P2T.coeffs)        
        dotP2[knext] = s1P2[knext] + s2P2[knext]             
        dott[knext] = n2Qs[knext]
        
        #@show "That's ok"
        Q1T.coeffs[knext + 1] = dotQ1[knext]/knext        
        P1T.coeffs[knext + 1] = dotP1[knext]/knext        
        Q2T.coeffs[knext + 1] = dotQ2[knext]/knext        
        P2T.coeffs[knext + 1] = dotP2[knext]/knext        
        tT.coeffs[knext + 1] = dott[knext]/knext
    end
    
    return Taylor1[tT, Q1T, P1T, Q2T, P2]
end

jetundriven (generic function with 1 method)

In [8]:
function undrivenIntegration(x10::Float64, px10::Float64, time_max::Float64, jetEqs::Function )
    
    Q₁, Q₂, P₁, P₂ = condini(x10, px10)
    τ₀ = 0.0
    t0 = 0.0
    
    ene0 = regHam(Q₁, Q₂, P₁, P₂)
        
    Q1v, Q2v, P1v, P2v = Float64[], Float64[], Float64[], Float64[]
    tv, τv = Float64[], Float64[]
    DeneV= Float64[]
    push!(tv, t0)
    push!(τv, τ₀)
    push!(Q1v, Q₁)
    push!(P1v, P₁)
    push!(Q2v, Q₂)
    push!(P2v, P₂)
       
    push!(DeneV, zero(Float64))
    
    # This is the main loop; we include a minimum step size for security
    dτ = 1.0
    while dτ < time_max && dτ>1.0e-8
        #Preparamos el vector para integrar
        temp1 = Float64[t0, Q₁ , Q₂, P₁, P₂,]
        # Here we integrate
        dτ, (t, Q1, P1, Q2, P2) = taylorStepper( jetEqs, temp1);
        
        τ₀ += dτ
        push!(tv, t)
        push!(τv, τ₀)
        push!(Q1v, Q1)        
        push!(P1v, P1)        
        push!(Q2v, Q2)        
        push!(P2v, P2)
        eneEnd = regHam(Q1, P1, Q2, P2)
        
        dEne = abs(eneEnd)
        
        push!(DeneV, dEne)
        temp1 = Float64[t, Q1, P1, Q2, P2]
    end

    return τv, tv, Q1v, P1v, Q2v, P2v, DeneV
end

undrivenIntegration (generic function with 1 method)

Para la configuración de $Zee$ en 1D necesitamos que $y_1, y_2 = 0$; $p_{y_1}, p_{y_2} = 0$, además, necesitamos la condición de que la partícula 2 colisione con el núcleo, por lo que $x_2 = 0$ sobre las coordenadas $(\mathbf{P}, \mathbf{Q})$ implican que:

\begin{align}
    x_1 = x_{1_0} &\Leftrightarrow Q_{11}^2 - Q_{12}^2 = x_{1_0}\\
    y_1 = 0 &\Leftrightarrow 2Q_{11}Q_{12} = 0\\
    x_2 = 0 &\Leftrightarrow Q_{21}^2 - Q_{22}^2 = x_{2_0}\\
    y_2 = 0 &\Leftrightarrow 2Q_{21}Q_{22} = 0 \\
    p_{x_1} = px_{1_0} &\Leftrightarrow \frac{Q_{11}P_{11} - Q_{12}P_{12}}{2R_1^2} = px_{1_0}\\
    p_{y_1} = 0 &\Leftrightarrow \frac{Q_{12}P_{11} + Q_{11}P_{12}}{2R_1^2} = 0\\
    p_{x_2} = px_{2_0} &\Leftrightarrow \frac{Q_{21}P_{21} - Q_{22}P_{22}}{2R_2^2} = px_{2_0}\\
    p_{y_2} = 0 &\Leftrightarrow \frac{Q_{22}P_{21} + Q_{21}P_{22}}{2R_2^2} = 0
\end{align}


A partir de esto, podemos concluir que:

\begin{align}
    Q_{11} &= \sqrt{x_{1_0}}\\
    Q_{12} &= 0\\
    Q_{21} &= 0\\
    Q_{22} &= 0\\
    P_{11} &= \frac{2R_1^2px_{1_0}}{Q_{11}}\\
    P_{12} &= 0\\
    P_{22} &= 0
\end{align}

Para las condiciones sobre los momento $P_{21} = 0$ de la partícula 2, debemos considerar la condición sobre la energía del sistema (considerando el hamiltoniano regularizado $H^{SC}$), pues recordemos que se debe cumplir que $H = 0$. Usando las condiciones ya encontradas se halla que debe cumplirse que $|P_2|^2 = Z = 2$, entonces:

\begin{align}
    Q_{11} &= \sqrt{x_{1_0}}\\
    Q_{12} &= 0\\
    Q_{21} &= 0\\
    Q_{22} &= 0\\
    P_{11} &= \frac{2x_{1_0}p_{x_{1}}}{\sqrt{x_{1_0}}}\\
    P_{12} &= 0\\
    P_{21} &= \sqrt{Z}\\
    P_{22} &= 0
\end{align}


In [9]:
function condini(x10::Float64, px10::Float64)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = 2*x10*px10/sqrt(x10)    
    P2 = sqrt(Z)    
    return Q1, Q2, P1, P2
end

condini (generic function with 1 method)

In [10]:
condini(0.7, 0.4)

([0.83666,0.0],[0.0,0.0],[0.669328,0.0],[1.41421,0.0])

In [11]:
@time tv, τv, Q11v, Q12v, P11v, P12v, Q21v, Q22v, P21v, P22v, 
DeneV = undrivenIntegration(5.1, 1.4, 2.1, jetundriven)

LoadError: InterruptException: